In [1]:
library(tidyverse)

-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.1.0     v dplyr   1.0.5
v tidyr   1.1.3     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



In [2]:
df <- read.csv("archive/realdonaldtrump.csv")
df$date <- as.POSIXct(df$date, format="%Y-%m-%d %H:%M:%S")
#Encoding(df$content) <- "latin1"
tweets <- df %>% filter(date > as.POSIXct("2016-01-01 12:00:00", format="%Y-%m-%d %H:%M:%S")) %>% 
    mutate(content = sub("https", " https", content)) %>% 
    mutate(content = sub("pic.twitter", " pic.twitter", content)) %>%
    mutate(content = gsub("[A-Za-z]{1,5}[.][A-Za-z]{2,3}/[A-Za-z0-9]+\\b", "", content)) %>% 
    mutate(content = qdapRegex::rm_url(content)) %>%
    mutate(content = sub("pic.tw", "", content)) %>%
    mutate(content = gsub("<.*?>", "", content, fixed=TRUE)) 

tweets$content[1]

[1] "\" @ sprinklermanus: @ CNN @ realDonaldTrump they're spending millions but you're still going to win - go Donald Trump\""

In [3]:
for (tweet in tweets$content) {
    vec <- unlist(str_split(gsub("<.*?>", "", tweet), " "))
    s <- c(s, c(NA, vec[stringi::stri_enc_isascii(vec)]))
}

In [5]:
s1 <- s[1:(length(s)-2)]
s2 <- s[2:(length(s)-1)]
s3 <- s[3:length(s)]
strings <- data.frame(s1,s2,s3)

In [6]:
transmat <- (strings %>% group_by(s1,s2, s3)) %>% summarise(count = n()) %>% drop_na()
transmat  %>% arrange(desc(count))

`summarise()` has grouped output by 's1', 's2'. You can override using the `.groups` argument.



s1,s2,s3,count
<chr>,<chr>,<chr>,<int>
I,will,be,285
the,United,States,223
MAKE,AMERICA,GREAT,219
AMERICA,GREAT,AGAIN!,210
all,of,the,207
Thank,you,to,205
the,Fake,News,196
Great,State,of,138
#,Trump2016,,136


In [7]:
u <- sample(1:nrow(transmat),1)
start <- c(transmat[u,1], transmat[u,2])
out <- c()
num.sentences <- 0
while (TRUE) {
    out <- c(out, start[1])
    T <- (transmat %>% filter(s1 == start[1], s2 == start[2]))
    if (length(T$s3) == 0) {
        out <- c(out, start[2])
        break()
    }
    if (length(out) > 50) {
        out <- c(out, start[2])
        break()
    }
    start <- c(start[2], as.character(sample(T$s3, size = 1, prob = T$count/sum(T$count))))
}
paste(out, collapse = " ")

[1] "Dems let @ FoxNews past. Great job Heather!"